In [ ]:
!pip install pip==23.1.2
!pip install datasets pydub torchaudio fairseq
!sudo apt install libsox-dev

In [ ]:
!git clone https://github.com/yt-dlp/yt-dlp.git yt-dlp
%cd yt-dlp
!make yt-dlp
!sudo cp yt-dlp /usr/local/bin/

In [ ]:
cookies = """
# Netscape HTTP Cookie File
# This file is generated by youtube-dl.  Do not edit.

.google.com	TRUE	/	TRUE	1748961130	AEC	wwwwwww
.youtube.com	TRUE	/	TRUE	1733432221	GPS	1
.youtube.com	TRUE	/	TRUE	1749240520	NID	wwwwwww
.youtube.com	TRUE	/	TRUE	1767989250	PREF	tz=wwwww
.youtube.com	TRUE	/	TRUE	0	SOCS	CAI
.youtube.com	TRUE	/	TRUE	1748960410	VISITOR_INFO1_LIVE	wwwww
.youtube.com	TRUE	/	TRUE	1748960410	VISITOR_PRIVACY_METADATA	wwwww
.youtube.com	TRUE	/	TRUE	0	YSC	wwwww
"""

In [ ]:
files_to_download_extended = []

In [ ]:
from __future__ import unicode_literals
import yt_dlp

#    'format': '(bestvideo[width>=1080][ext=mp4]/bestvideo)+bestaudio/best', #Ensures best settings
ydl_opts = {
    'format': 'best', #Ensures best settings
    'postprocessors': [
      {
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
          'preferredquality': '320',
          'nopostoverwrites': True,  # Prevents overwriting files that already exist
      }
    ],
    "outtmpl":'/content/ytm/%(title)s.%(ext)s',
    'cookiefile': '/content/cookies.txt',
    "retries": 10,
    "fragment_retries": 10,
    "sleep_interval": 5,
    "http_headers": {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(files_to_download_extended)

In [ ]:
import os
from pydub import AudioSegment
import time

import sys
import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import soundfile as sf
import torch
import torchaudio
from fairseq.data.audio.audio_utils import get_features_or_waveform

In [ ]:
def create_wav_meta_tsv(audio_dir, output_dir):
    with open(output_dir, "a") as f:
        for filename in os.listdir(audio_dir):
            if filename.endswith(".wav"):
                audio_path = os.path.join(audio_dir, filename)
                try:
                    audio = AudioSegment.from_wav(audio_path)
                    duration = len(audio) / 1000.0  # Convert milliseconds to seconds
                    f.write(f"{audio_path}\t{duration}\t{int(time.time() + duration)}\n")
                except Exception as e:
                    print(f"Error processing {audio_path}: {e}")

In [ ]:
class MfccFeatureReader(object):
    def __init__(self, sample_rate):
        self.sample_rate = sample_rate

    def read_audio_mp3(self, path):
      audio = AudioSegment.from_mp3(path)  # Load MP3 file
      audio = audio.set_channels(1)  # mono
      audio = audio.set_frame_rate(self.sample_rate)
      #waveform = np.array(audio.get_array_of_samples())
      wav_path = audio.export(f"{path}.wav", format="wav").name
      return wav_path

    def read_audio(self, path, ref_len=None):
        #wav_path = self.read_audio_mp3(path)
        wav_path = path
        wav = get_features_or_waveform(wav_path, need_waveform=True, use_sample_rate=self.sample_rate)
        if ref_len is not None and abs(ref_len - len(wav)) > 160:
            print(f"ref {ref_len} != read {len(wav)}")
        return wav

    def get_feats(self, path, ref_len=None):
        x = self.read_audio(path, ref_len=ref_len)
        with torch.no_grad():
            x = torch.from_numpy(x).float()
            x = x.view(1, -1)

            mfccs = torchaudio.compliance.kaldi.mfcc(
                waveform=x,
                sample_frequency=self.sample_rate,
                use_energy=False,
            )  # (time, freq)
            mfccs = mfccs.transpose(0, 1)  # (freq, time)
            deltas = torchaudio.functional.compute_deltas(mfccs)
            ddeltas = torchaudio.functional.compute_deltas(deltas)
            concat = torch.cat([mfccs, deltas, ddeltas], dim=0)
            concat = concat.transpose(0, 1).contiguous()  # (freq, time)
            return concat

In [ ]:
def get_path_iterator(tsv):
    with open(tsv, "r") as f:
        lines = [line.rstrip() for line in f]
        def iterate():
            for line in lines:
                subpath, nsample, idx_id = line.split("\t")
                yield f"{subpath}", int(float(nsample)), idx_id
    return iterate, len(lines)

def dump_feature_n(reader, generator, num, feat_dir):
    iterator = generator()
    parquet_writer = None
    features = []
    lengths = []
    idx_val = []

    feat_path = f"{feat_dir}mfcc_features_1.parquet"

    for path, nsample, idx_id in tqdm.tqdm(iterator, total=num):
        feat = reader.get_feats(path, nsample)
        feat_np = feat.cpu().numpy()
        features.append(feat_np)
        lengths.append(len(feat))
        idx_val.append(idx_id)

    table = pa.table({
        'features': pa.array(features, type=pa.list_(pa.float64())),
        'duration_sec': pa.array(lengths, type=pa.int32()),
        'indx_id':  pa.array(lengths, type=pa.int32()),
      })
    parquet_writer = pq.ParquetWriter(feat_path, table.schema)
    parquet_writer.write_table(table)

    if parquet_writer is not None:
        parquet_writer.close()

def dump_duration(generator, num):
    iterator = generator()
    duration = []
    for path, nsample, idx_id in tqdm.tqdm(iterator, total=num):
        duration.append(nsample)
    return duration

In [ ]:
meta_file_path = "/content/voice_meta/train.tsv"
audio_dir = "/content/ytm"  # Directory containing wav files
dump_mfcc_feat_dir = "/content/km/"
sample_rate = 16000
reader = MfccFeatureReader(sample_rate)

create_wav_meta_tsv(audio_dir, meta_file_path)
generator, num = get_path_iterator(f"{meta_file_path}")
dump_feature_n(reader, generator, num, dump_mfcc_feat_dir)

In [ ]:
generator, num = get_path_iterator(f"{meta_file_path}")
duration_lst = dump_duration(generator, num)
print(f"{sum(duration_lst)/(60*60)} hrs")

In [ ]:
import soundfile as sf
import librosa
import pandas as pd

def process_audio_features(mfcc_features_s1, target_sample_rate=16000):
    data = []
    audio_files = []

    for line in mfcc_features_s1.strip().split("\n"):
        file_path, duration, idx_key = line.split("\t")
        audio_files.append({
            "file_path": file_path,
            "duration_sec": float(duration),
            "indx_id": idx_key
        })

    for audio_info in audio_files:
        file_path = audio_info["file_path"]
        idx_key = audio_info["indx_id"]
        duration_sec = audio_info["duration_sec"]

        try:
            audio, sample_rate = sf.read(file_path)
            audio_features = librosa.resample(y=audio.T if audio.ndim > 1 else audio, orig_sr=sample_rate, target_sr=target_sample_rate)
            data.append({
                "wav_features": audio_features.tolist(),
                "indx_id": idx_key,
                "duration_sec": duration_sec
            })
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

    df = pd.DataFrame(data)
    return df

In [ ]:
output_parquet_file = "wav_features_s1.parquet"
audio_meta_file = """audio.wav 7455    11114445""" # or read meta_file_path
df = process_audio_features(audio_meta_file)
df.to_parquet(output_parquet_file, engine="pyarrow", index=False)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="/content/km/mfcc_features_1.parquet",
    path_in_repo="mfcc_features_s1.parquet",
    repo_id="",
    repo_type="dataset",
)